1. Loads csv as pandas dataframe
2. Converts each line of the dataframe to tensor
3. Creates _MapDataset obj (same as obj recieved when loading sample data (TensorFlow2))
4. Preprocess data -> needs to convert to array and back to tensor to make operations -> CHANGE!! preprocess the csv before to simplify
5. Embed both towers and train the model
6. Obtain results

In [9]:
import tqdm as notebook_tqdm
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

import pandas as pd
import numpy as np
import tensorflow as tf
import pandas as pd
import functions as f

from typing import Dict, Text
import pprint

In [10]:
#we get the customers and article data
# customers_path = '../00 - Data/customers/customers.csv'

transactions_path = '../00 - Data/transactions/transactions_train_short.csv'
articles_path = '../00 - Data/articles/articles.csv'

In [20]:
articles = pd.read_csv(articles_path)
len(articles)

105542

In [13]:
transactions = pd.read_csv(transactions_path)
transactions.drop('Unnamed: 0', axis=1, inplace=True)
transactions.drop('week', axis=1, inplace=True)
transactions.drop('t_dat',axis=1, inplace=True)
transactions.drop('price',axis=1, inplace=True)
transactions.drop('sales_channel_id',axis=1, inplace=True)

transactions.reset_index(drop=True)


,customer_id,article_id
0,200292573348128,880777001
1,200292573348128,784332002
2,200292573348128,827968001
3,200292573348128,599580086
4,248294615847351,720504008
...,...,...
2473339,18439937050817258297,891591003
2473340,18439937050817258297,869706005
2473341,18440902715633436014,918894002
2473342,18440902715633436014,761269001


In [14]:
transactions.to_csv('../00 - Data/transactions/transactions_train_short2.csv', index=False)
transactions

,customer_id,article_id
0,200292573348128,880777001
1,200292573348128,784332002
2,200292573348128,827968001
3,200292573348128,599580086
4,248294615847351,720504008
...,...,...
2473339,18439937050817258297,891591003
2473340,18439937050817258297,869706005
2473341,18440902715633436014,918894002
2473342,18440902715633436014,761269001


In [15]:
transactions.dtypes

customer_id    uint64
article_id      int64
dtype: object

In [18]:
articles = pd.read_csv(articles_path)
articles
articles = articles["article_id"]

articles.to_csv('../00 - Data/articles/articles2.csv', index=False)


In [19]:
articles

0         108775015
1         108775044
2         108775051
3         110065001
4         110065002
            ...    
105537    953450001
105538    953763001
105539    956217002
105540    957375001
105541    959461001
Name: article_id, Length: 105542, dtype: int64

In [7]:
transactions_path = '../00 - Data/transactions/transactions_train_short2.csv'

In [ ]:
columns_to_keep = ['article_id', 'customer_id']
data = pd.read_csv(transactions_path, usecols=columns_to_keep)

# Convert pandas DataFrame to TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices((dict(data),))

In [ ]:
batch_size  = 64
articles = tf.data.TextLineDataset(articles_path).skip(1)  # Skip the header
articles = articles.map(f.parse_csv_line_a, num_parallel_calls=tf.data.experimental.AUTOTUNE) # creates _ParallelMapDataset
# transactions = transactions.batch(batch_size)  # Choose an appropriate batch size -> creates _BatchDataset

transactions = tf.data.TextLineDataset(transactions_path).skip(1)  # Skip the header
transactions = transactions.map(f.parse_csv_line_t, num_parallel_calls=tf.data.experimental.AUTOTUNE) # creates _ParallelMapDataset
# transactions = transactions.batch(batch_size)  # Choose an appropriate batch size -> creates _BatchDataset

print(articles.element_spec)
print(transactions.element_spec)

In [32]:
# limiting the features for testing
transactions = transactions.map(lambda x:{'customer_id':x['customer_id'],'article_id':x['article_id']})
articles = articles.map(lambda x:{'article_id':x['article_id']})
print(articles.element_spec)
print(transactions.element_spec)

{'article_id': TensorSpec(shape=(), dtype=tf.string, name=None)}
{'customer_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'article_id': TensorSpec(shape=(), dtype=tf.float32, name=None)}


In [33]:
customer_id_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
customer_id_vocabulary.adapt(transactions.map(lambda x: x['customer_id']))

article_id_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
article_id_vocabulary.adapt(articles.map(lambda x: x['article_id']))


# # t_dat_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
# # t_dat_vocabulary.adapt(transactions.map(lambda x: x['t_dat']))

# # Convert the symbolic tensor to a Python list for article_id
# article_id_data = [x['article_id'].numpy() for x in articles]

# # Adapt the article_id vocabulary
# article_id_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
# article_id_vocabulary.adapt(article_id_data)

InvalidArgumentError: Graph execution error:

Detected at node DecodeCSV defined at (most recent call last):
<stack traces unavailable>
Expect 5 fields but have 7 in record 0
	 [[{{node DecodeCSV}}]]
	 [[IteratorGetNext]] [Op:__inference_adapt_step_1022]

In [14]:
class hmModel(tfrs.Model):
 
  def __init__(
      self,
      user_model: tf.keras.Model,
      article_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()
 
    # Set up user and movie representations.
    self.user_model = user_model
    self.article_model = article_model
 
    # Set up a retrieval task.
    self.task = task
 
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.
 
    user_embeddings = self.user_model(features["customer_id"])
    article_model = self.article_model(features["article_id"])
 
    return self.task(user_embeddings, article_model)


In [ ]:
 # Set up user and movie representations.
    self.user_model = tf.keras.Sequential([
      # We first turn the raw user ids into contiguous integers by looking them
      # up in a vocabulary.
      tf.keras.layers.experimental.preprocessing.StringLookup(
          max_tokens=num_unique_users),
      # We then map the result into embedding vectors.
      tf.keras.layers.Embedding(num_unique_users, embedding_dim)
    ])

In [16]:
users_model = tf.keras.Sequential([transactions,tf.keras.layers.Embedding(customer_id_vocabulary.vocab_size(),64)])
article_model = tf.keras.Sequential([articles,tf.keras.layers.Embedding(article_id_vocabulary.vocab_size(),64)])
 
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(article_id_data))

TypeError: The added layer must be an instance of class Layer. Received: layer=<_MapDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'article_id': TensorSpec(shape=(), dtype=tf.float32, name=None), 't_dat': TensorSpec(shape=(), dtype=tf.string, name=None)}> of type <class 'tensorflow.python.data.ops.map_op._MapDataset'>.

In [20]:
model = hmModel(users_model,article_model,task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))
model.fit(transactions.batch(4096), epochs=3)

Epoch 1/3


AttributeError: in user code:

    File "/home/pablo/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/home/pablo/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/pablo/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "/home/pablo/.local/lib/python3.10/site-packages/tensorflow_recommenders/models/base.py", line 68, in train_step
        loss = self.compute_loss(inputs, training=True)
    File "/tmp/ipykernel_7682/341823061.py", line 23, in compute_loss
        return self.task(user_embeddings, article_model)
    File "/home/pablo/.local/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceb

In [ ]:
recommends = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
recommends.index_from_dataset(articles.batch(100).map(lambda title: (title, model.article_model(title))))
 
id_ = input('Enter the user_id: ')
_, titles = recommends(np.array([str(id_)]))
print('Top recommendation for user',id_,titles[0, :3])